In [80]:
from serpapi import GoogleSearch
import requests
from bs4 import BeautifulSoup
import os
from IPython.display import display, Markdown
from http.client import responses as http_responses

from langchain.llms import OpenAI
from langchain.document_loaders.base import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain

In [82]:
search_query = "device that attaches to the soil and relays the plant's needs to the phone app directly"

llm = OpenAI(
    model="text-davinci-003",
    temperature=0,
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

search = GoogleSearch({
    "q": search_query,
    "location": "Mumbai, Maharashtra, India",
    "api_key": os.environ["SERPAPI_API_KEY"]
})

results = search.get_dict()

if "error" in results:
    print(f"Error: {results['error']}")
else:
    print(f"Number of organic results: {len(results['organic_results'])}")

vectordb = Chroma(embedding_function=OpenAIEmbeddings())

Number of organic results: 9


In [83]:
results_condensed = [(result['title'], result['link']) for result in results['organic_results']]

content_p = ""
count_p = 0

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

for title, link in results_condensed:
    print(f"Title: {title}")
    print(f"Link: {link}")

    response = requests.get(link)
    print(f"Response code: {response.status_code}")
    print(f"Reponse Message: {http_responses[response.status_code]}")
    if response.status_code != 200:
        print()
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    webpage = ""
    webpage += f'## {title}' + "\n"

    content_p += f'## {title}' + "\n"
    for p in soup.find_all('p'):
        paragraph = p.get_text(separator=' ')

        if len(paragraph) > 100:
            webpage += paragraph
            content_p += paragraph
            content_p += "\n\n"
            count_p += 1

    doc = text_splitter.create_documents(texts=[content_p], metadatas=[{"source": link, "title": title}])
    ids = vectordb.add_documents(documents=[*doc])
    print(f"Added {len(ids)} documents to the database")
    print()

    content_p += "\n-------------------------------------------------------------------------------------\n"

Title: Plug in your plants with these 15 connected garden ...
Link: https://www.cnet.com/pictures/plug-in-your-plants-with-these-connected-garden-gadgets-pictures/
Response code: 200
Reponse Message: OK
Added 7 documents to the database

Title: Plant Link: Listen to your plants by Oso Technologies
Link: https://www.kickstarter.com/projects/1387729422/plant-link-listen-to-your-plants
Response code: 403
Reponse Message: Forbidden

Title: Amazon.com: Plant Monitor Soil Test Kit,Flower Care ...
Link: https://www.amazon.com/WANFEI-Intelligent-Bluetooth-Fertility-Temperature/dp/B09PV3T356
Response code: 503
Reponse Message: Service Unavailable

Title: Arduino based Automatic Plant Irrigation System with ...
Link: https://circuitdigest.com/microcontroller-projects/arduino-automatic-plant-watering-system
Response code: 200
Reponse Message: OK
Added 27 documents to the database

Title: Wifi Plant Watering Sensor System
Link: https://www.postscapes.com/wireless-plant-sensors/
Response code: 200


Created a chunk of size 1056, which is longer than the specified 1000


Response code: 200
Reponse Message: OK
Added 47 documents to the database

Title: Automated Plant Watering System
Link: https://www.ijert.org/research/automated-plant-watering-system-IJERTCONV5IS18012.pdf


Created a chunk of size 1056, which is longer than the specified 1000


Response code: 200
Reponse Message: OK
Added 47 documents to the database

Title: Resilient and Responsible Smart Cities
Link: https://books.google.com/books?id=aDV9EAAAQBAJ&pg=PA88&lpg=PA88&dq=device+that+attaches+to+the+soil+and+relays+the+plant's+needs+to+the+phone+app+directly&source=bl&ots=_3bwo3kOqx&sig=ACfU3U3aiGojD36qkgZy3o2B5Z3DR7_BnA&hl=en&sa=X&ved=2ahUKEwikusWByq-AAxX6MVkFHYqsDjkQ6AF6BAhAEAM


Created a chunk of size 1056, which is longer than the specified 1000


Response code: 200
Reponse Message: OK
Added 47 documents to the database



In [ ]:
# display(Markdown(content_p))

In [90]:
retriever = vectordb.as_retriever()

qa_chain = RetrievalQAWithSourcesChain.from_chain_type(llm=llm,
                                                       chain_type="stuff",
                                                       retriever=retriever,)

In [91]:
query = "Name 3 devices that help you monitor your plant's health, along with unique features of each"

llm_response = qa_chain(query)

print(llm_response['answer'])
print(llm_response['sources'])

 Three devices that help you monitor your plant's health are the Arduino Automatic Plant Watering System, Wireless Plant Sensors, and an Automatic Plant Watering System Using a Microbit. The Arduino Automatic Plant Watering System can be programmed to water plants based on the weather, the Wireless Plant Sensors can take readings from the soil, and the Automatic Plant Watering System Using a Microbit can attach to the soil and relay the plant's needs to a phone app directly.


Circuit Digest, "Arduino Automatic Plant Watering System", https://circuitdigest.com/microcontroller-projects/arduino-automatic-plant-watering-system
Postscapes, "Wireless Plant Sensors", https://www.postscapes.com/wireless-plant-sensors/
Instructables, "Automatic Plant Watering System Using a Microbit", https://www.instructables.com/Automatic-Plant-Watering-System-Using-a-Microbit/
Google Books, "Automatic Plant Watering System Using a Microbit", https://books.google.com/books?id=aDV9EAAAQBAJ&pg=


In [96]:
query = "Name 3 devices that help you monitor your plant's health"

print(vectordb.similarity_search(query, k=5)[0].page_content)

The first type of device we'll look at: plant sensors. You place these devices into the dirt near your plant, where they keep a close eye on growing conditions and send you recommendations accordingly. The Koubachi Wi-Fi Plant Sensor is the most expensive one we've seen -- it's $100 for the indoor version and $130 for the weather-proofed one -- but it measures moisture, light and temperature and sends that all directly to the cloud via Wi-Fi, letting you check in on your plant from anywhere.

The $60 Parrot Flower Power brings the price tag down quite a bit. It also measures everything Koubachi does, and adds readings for fertilizer. However, since it's limited to Bluetooth, it doesn't have Koubachi's reach, meaning you'll need to be relatively close to your plants in order to get up-to-date readings.
